In [1]:
import sys
sys.path.append('../')

In [2]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

In [3]:
from representer.representer_model import RepresenterModel

In [4]:
mnist_dataset = tf.keras.datasets.mnist
(train_images, train_labels), (test_images, test_labels) = mnist_dataset.load_data()

train_images = train_images / 255.0
test_images = test_images / 255.0

categorical_train_labels = tf.keras.utils.to_categorical(train_labels)
categorical_test_labels = tf.keras.utils.to_categorical(test_labels)

tf.keras.backend.set_floatx("float64")

In [5]:
model = tf.keras.Sequential(
    [
        tf.keras.layers.Flatten(input_shape=(28, 28)),
        tf.keras.layers.Dense(10, use_bias=False, kernel_regularizer="l2"),
    ]
)

model.compile(
    optimizer="adam",
    loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
    metrics=["accuracy"],
)

In [6]:
model.fit(train_images, categorical_train_labels, epochs=10, validation_split=0.1, shuffle=False)

Epoch 1/10
1688/1688 [==============================] - 1s 845us/step - loss: 0.7978 - accuracy: 0.8560 - val_loss: 0.6323 - val_accuracy: 0.9065
Epoch 2/10
1688/1688 [==============================] - 1s 744us/step - loss: 0.7009 - accuracy: 0.8816 - val_loss: 0.6321 - val_accuracy: 0.9063
Epoch 3/10
1688/1688 [==============================] - 1s 742us/step - loss: 0.7008 - accuracy: 0.8817 - val_loss: 0.6322 - val_accuracy: 0.9063
Epoch 4/10
1688/1688 [==============================] - 1s 744us/step - loss: 0.7009 - accuracy: 0.8818 - val_loss: 0.6322 - val_accuracy: 0.9063
Epoch 5/10
1688/1688 [==============================] - 1s 777us/step - loss: 0.7009 - accuracy: 0.8818 - val_loss: 0.6322 - val_accuracy: 0.9063
Epoch 6/10
1688/1688 [==============================] - 1s 753us/step - loss: 0.7009 - accuracy: 0.8818 - val_loss: 0.6322 - val_accuracy: 0.9063
Epoch 7/10
1688/1688 [==============================] - 1s 753us/step - loss: 0.7009 - accuracy: 0.8818 - val_loss: 0.6322 -

In [7]:
feature_model = model.get_layer(index=0)
prediction_network = model.get_layer(index=1)

In [8]:
l2 = 0.01
num_training_points = 54000

train_x = train_images[0:1]
train_y = categorical_train_labels[0:1]
test_x = test_images[0:1]
test_y = categorical_test_labels[0:1]

In [9]:
with tf.GradientTape() as tape:
    pred = model(train_x)
    loss = model.loss(train_y, pred)
alpha = - tape.gradient(loss, pred) / (2*l2*num_training_points) # shape=(1, 10)

training_features = feature_model(train_x) # shape=(1, 784)
test_features = feature_model(test_x) # shape(1, 784)

representer_val = tf.math.reduce_sum(tf.math.multiply(training_features, test_features)) * alpha
representer_val

<tf.Tensor: shape=(1, 10), dtype=float64, numpy=
array([[-8.51893332e-04, -1.00680159e-04, -6.06192453e-04,
        -8.41183413e-03, -3.08212985e-05,  1.27211042e-02,
        -4.26574232e-04, -9.70710137e-04, -1.01438233e-03,
        -3.08016127e-04]])>

In [10]:
representer_model = RepresenterModel(
    feature_model,
    prediction_network,
    train_images,
    categorical_train_labels,
    test_images,
    model.loss,
    num_training_points=54000
)

In [11]:
representer_model.get_representer_value(0, 0)

<tf.Tensor: shape=(1, 10), dtype=float64, numpy=
array([[-8.51893332e-04, -1.00680159e-04, -6.06192453e-04,
        -8.41183413e-03, -3.08212985e-05,  1.27211042e-02,
        -4.26574232e-04, -9.70710137e-04, -1.01438233e-03,
        -3.08016127e-04]])>